This notebook converts xml files located in s3 to a dictionary with the following format, and then puts the dictionary back into the s3 bucket: </br>
{congress_number: {bill_id1: bill_text1}, {bill_id2: bill_text2}...}

In [1]:
pip install boto3

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 131 kB 34.9 MB/s eta 0:00:01
     |████████████████████████████████| 79 kB 20.5 MB/s  eta 0:00:01
     |████████████████████████████████| 8.4 MB 79.0 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
import requests
import re
import time
import json
import sys
import boto3

In [2]:
s3 = boto3.client('s3')
s3_resource = boto3.resource('s3')
bucket_resource = s3_resource.Bucket('macs30123-bills')

In [5]:
all_files = bucket_resource.objects.all()

In [3]:
file_names = []

In [13]:
# congress = "113"
# congress = "114"
# congress = "116"
congress = "115"

In [6]:
for obj in all_files:
    if re.search(r"BILLS-115", obj.key):
        file_names.append(obj.key)

In [8]:
lengths = {'115': 15534, '116': 18998, '114': 14784, '113': 12504}

15534

In [7]:
# check that all bills made it this far
len(file_names) == lengths[congress]

15534

In [9]:
all_bills = {congress: {}}

In [10]:
l = len(file_names)

In [11]:
for num, file in enumerate(file_names):
    print(f"{(num/l)*100}%")
    bill_113 = s3.get_object(Bucket="macs30123-bills", Key=file)
    billid = re.search(r"BILLS-115-(\d)-(.+)\/(.+)\.xml", file).groups()[2]
    bill_113_body = bill_113['Body'].read()
    bill_113_decode = bill_113_body.decode('utf8')
    all_bills[congress][billid] = bill_113_decode

0.0%
0.006437491953135059%
0.012874983906270118%
0.019312475859405175%
0.025749967812540235%
0.03218745976567529%
0.03862495171881035%
0.04506244367194541%
0.05149993562508047%
0.057937427578215524%
0.06437491953135058%
0.07081241148448565%
0.0772499034376207%
0.08368739539075576%
0.09012488734389082%
0.09656237929702588%
0.10299987125016094%
0.10943736320329599%
0.11587485515643105%
0.12231234710956612%
0.12874983906270115%
0.13518733101583622%
0.1416248229689713%
0.14806231492210634%
0.1544998068752414%
0.16093729882837646%
0.16737479078151152%
0.17381228273464658%
0.18024977468778164%
0.1866872666409167%
0.19312475859405176%
0.19956225054718682%
0.20599974250032188%
0.21243723445345694%
0.21887472640659197%
0.22531221835972703%
0.2317497103128621%
0.23818720226599718%
0.24462469421913224%
0.2510621861722673%
0.2574996781254023%
0.2639371700785374%
0.27037466203167243%
0.2768121539848075%
0.2832496459379426%
0.28968713789107764%
0.2961246298442127%
0.30256212179734776%
0.308999613750

In [14]:
len(all_bills[congress])

15534

In [15]:
bucket_resource.put_object(Key=f'{congress}_bills', Body=json.dumps(all_bills))

s3.Object(bucket_name='macs30123-bills', key='115_bills')